# LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import xgboost
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
conexion=sqlite3.connect("C:/Users/Administrador/OneDrive - ESIC/Proyecto Big Data I/TO Hoteles/Data/hotel_bookings_clean.db")
df=pd.read_sql_query("select * from hotel_bookings_clean",conexion)
conexion.close()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87370 entries, 0 to 87369
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87370 non-null  object 
 1   is_canceled                     87370 non-null  int64  
 2   lead_time                       87370 non-null  int64  
 3   arrival_date_year               87370 non-null  int64  
 4   arrival_date_month              87370 non-null  object 
 5   arrival_date_week_number        87370 non-null  int64  
 6   arrival_date_day_of_month       87370 non-null  int64  
 7   stays_in_weekend_nights         87370 non-null  int64  
 8   stays_in_week_nights            87370 non-null  int64  
 9   adults                          87370 non-null  int64  
 10  children                        87370 non-null  float64
 11  babies                          87370 non-null  int64  
 12  meal                            

Obtener solo las variables categóricas y pasarlas a una lista

In [9]:
categoricas=df.select_dtypes(include=object).columns.tolist()

In [10]:
df[categoricas].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87370 entries, 0 to 87369
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   hotel                    87370 non-null  object
 1   arrival_date_month       87370 non-null  object
 2   meal                     87370 non-null  object
 3   country                  87370 non-null  object
 4   market_segment           87370 non-null  object
 5   distribution_channel     87370 non-null  object
 6   reserved_room_type       87370 non-null  object
 7   assigned_room_type       87370 non-null  object
 8   deposit_type             87370 non-null  object
 9   customer_type            87370 non-null  object
 10  reservation_status       87370 non-null  object
 11  reservation_status_date  87370 non-null  object
dtypes: object(12)
memory usage: 8.0+ MB


In [6]:
ohe = OneHotEncoder(sparse_output=False, drop=None)
encoded= ohe.fit_transform(df[categoricas])
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(categoricas))

In [7]:
print(encoded_df)

       hotel_City Hotel  hotel_Resort Hotel  arrival_date_month_April  \
0                   0.0                 1.0                       0.0   
1                   0.0                 1.0                       0.0   
2                   0.0                 1.0                       0.0   
3                   0.0                 1.0                       0.0   
4                   0.0                 1.0                       0.0   
...                 ...                 ...                       ...   
87365               1.0                 0.0                       0.0   
87366               1.0                 0.0                       0.0   
87367               1.0                 0.0                       0.0   
87368               1.0                 0.0                       0.0   
87369               1.0                 0.0                       0.0   

       arrival_date_month_August  arrival_date_month_December  \
0                            0.0                          

In [8]:
numerical_cols = df.select_dtypes(exclude='object').columns.drop('adr')
df_final = pd.concat([df[numerical_cols], encoded_df, df['adr']], axis=1)

In [9]:
print(df_final)

       is_canceled  lead_time  arrival_date_year  arrival_date_week_number  \
0                0        342               2015                        27   
1                0        737               2015                        27   
2                0          7               2015                        27   
3                0         13               2015                        27   
4                0         14               2015                        27   
...            ...        ...                ...                       ...   
87365            0         23               2017                        35   
87366            0        102               2017                        35   
87367            0         34               2017                        35   
87368            0        109               2017                        35   
87369            0        205               2017                        35   

       arrival_date_day_of_month  stays_in_weekend_nights  \
0 

# ENTRENAMIENTO DEL MODELO

Vamos a dividir los datos a la mitad. Una parte para entrenarlos y otra para validarlos.

In [10]:
x = df_final.drop('adr', axis=1)
y = df_final['adr']

In [11]:
x_train, x_test, y_train, y_test =train_test_split (x, y, test_size=0.2, random_state=42)

In [65]:
x_train.shape, x_test.shape

((69896, 1185), (17474, 1185))

 XGBOOST

In [13]:
xgb =xgboost.XGBRegressor()

In [14]:
parametros = {
    'n_estimators': [500],      # ¿Cuántos árboles usar?
    'learning_rate': [0.1],    # ¿Qué tan rápido aprende?
    'max_depth': [9],                 # ¿Qué tan profundos son los árboles?
    'min_child_weight': [5],
    'random_state': [42],             # Semilla para reproducibilidad
}


In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
clf = GridSearchCV(
    estimator=xgb,
    param_grid=parametros,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=2
)

In [17]:
clf.fit(x_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=500, random_state=42; total time=  15.7s
[CV] END learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=500, random_state=42; total time=  15.3s
[CV] END learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=500, random_state=42; total time=  15.3s
[CV] END learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=500, random_state=42; total time=  15.1s
[CV] END learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=500, random_state=42; total time=  15.2s


,estimator,"XGBRegressor(...ree=None, ...)"
,param_grid,"{'learning_rate': [0.1], 'max_depth': [9], 'min_child_weight': [5], 'n_estimators': [500], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'reg:squarederror'


In [18]:
clf.best_score_

np.float64(-719.7691384106499)

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
best_xgb= clf.best_estimator_

In [21]:
y_preds = best_xgb.predict(x_test)

In [22]:
comp=pd.DataFrame({"real": y_test, "preds": y_preds})

In [23]:
comp.head(20)

,real,preds
9342,110.00,104.736633
20503,132.60,126.512321
51308,108.00,105.179665
48995,99.00,101.936089
68717,127.80,133.988876
82367,170.00,162.521973
38828,85.85,92.761406
2630,32.62,43.077190
84511,170.00,155.344955
33505,340.00,336.590393


In [24]:
comp['error'] = comp['real'] - comp['preds']
comp['error_abs'] = abs(comp['error'])
comp['error_pct'] = (comp['error_abs'] / comp['real']) * 100

In [25]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [26]:
# 5. Estadísticas del error
print("\n" + "=" * 60)
print("ESTADÍSTICAS DEL ERROR")
print("=" * 60)
print(f"Error medio: {comp['error'].mean():.2f}")
print(f"Error absoluto medio (MAE): {comp['error_abs'].mean():.2f}")
print(f"Error porcentual medio: {comp['error_pct'].mean():.2f}%")
print(f"RMSE: {np.sqrt(mean_squared_error(comp['real'], comp['preds'])):.2f}")
print(f"R²: {r2_score(comp['real'], comp['preds']):.4f}")



ESTADÍSTICAS DEL ERROR
Error medio: 0.06
Error absoluto medio (MAE): 10.12
Error porcentual medio: inf%
RMSE: 17.33
R²: 0.8927
